In [6]:
%load_ext dotenv
%dotenv

In [7]:
import boto3, json, sagemaker, time
import torch
import os
from sagemaker import get_execution_role
from transformers import AutoProcessor

/Users/jaredpangallozzi/PycharmProjects/llm_image_captioner/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
sess = boto3.Session()
sm = sess.client("sagemaker")
sm_client = boto3.client(service_name="sagemaker")
sagemaker_session = sagemaker.Session(boto_session=sess)
role = get_execution_role()
client = boto3.client("sagemaker-runtime")

In [21]:
triton_image_uri = "{account_id}.dkr.ecr.{region}.amazonaws.com/sagemaker-tritonserver:22.05-py3".format(
    account_id=os.environ["AWS_ACCOUNT_ID"], region="us-east-1"
)

In [10]:
# Our LLM, Llama, was downloaded via https://github.com/facebookresearch/llama?fbclid=IwAR3DJH6IFhcRGQJGRjUFFV1aXMjGJxXqMy5O_A9Hs1aNrl3Tdm5bUIHfSrA
# since it was easier to run Meta's provided shell script than download from Huggingface. The 7billion parameter model was chosen.

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [13]:
from transformers import BlipConfig, BlipForConditionalGeneration, AutoProcessor

processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

file_path = "../models/blip/1/model.pt"

torch.save(model, file_path)
# model = torch.load(file_path)

In [18]:
# Archive model and upload to S3
! tar -C ../models -czf ../blip.tar.gz blip
model_uri = sagemaker_session.upload_data(path="../blip.tar.gz", key_prefix="models")

In [23]:
print(model_uri)

s3://sagemaker-us-east-1-295763662460/models/blip.tar.gz


In [19]:
sm_model_name = "blip-pt-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

container = {
    "Image": triton_image_uri,
    "ModelDataUrl": model_uri,
    "Environment": {"SAGEMAKER_TRITON_DEFAULT_MODEL_NAME": "blip"},
}

create_model_response = sm_client.create_model(
    ModelName=sm_model_name, ExecutionRoleArn=role, PrimaryContainer=container
)

print("Model Arn: " + create_model_response["ModelArn"])


s3://sagemaker-us-east-1-295763662460/models/blip.tar.gz


In [20]:
## This part is commented out since the model is too big for 

# bs = 1
# width = 256
# height = 256
# channels = 3
# dummy_inputs = torch.rand(bs, channels, width, height).to(device)
# model = model.eval()
# model.to(device)

# traced_model = torch.jit.trace(model, dummy_inputs)